In [ ]:
!pip install transformers

In [ ]:
!pip install wandb

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-qp9zfe4b/kobert-tokenizer_a9542e2b80004317ad0400209333a981
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-qp9zfe4b/kobert-tokenizer_a9542e2b80004317ad0400209333a981
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=badd5a4ff9177da8a329463adfda3695ece9963fca855d9ab3c6898b91859c24
  Stored in directory: /tmp/pip-ephem-wheel-cache-bz77c1vr/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [7]:
model = BertModel.from_pretrained('skt/kobert-base-v1')

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [8]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   

In [11]:
df = pd.read_csv("concat.csv")

In [12]:
df.dropna(inplace=True)

In [13]:
label_dict = {'정치': 0, '사회': 1, '경제': 2, '국제': 3}
df['label'] = df['label'].map(label_dict)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42, stratify=df['label'])

In [38]:
class BERTDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = self.texts.iloc[idx]
    label = self.labels.iloc[idx]

    tokens = self.tokenizer(
        text,
        return_tensors='pt',     # 텐서로 반환
        truncation=True,         # 입력 시퀀스가 최대 길이를 초과할 때 자를지 자를지 여부 결정
        padding='max_length',
        max_length=self.max_len, # 최대 입력 길이
        add_special_tokens=True
    )

    input_ids = tokens['input_ids'].flatten() # 2D -> 1D
    attention_mask = tokens['attention_mask'].flatten()
    token_type_ids = tokens['token_type_ids'].flatten()
    # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환
    # input_ids : 토큰
    # attention_mask : 실제 단어가 존재하면 1, 패딩이면 0
    # token_type_ids : 문장을 구분하는 id, BERT는 입력으로 2개의 문장이 들어가지만 단일 문장인 경우에는 전부 0

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'token_type_ids': token_type_ids
        }, torch.tensor(label, dtype=torch.long)

In [16]:
max_len = 512
batch_size = 8

train_dataset = BERTDataset(X_train, y_train, tokenizer, max_len)
val_dataset = BERTDataset(X_val, y_val, tokenizer, max_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [44]:
# model에 입력값을 넣으면 결과값으로 last_hiden_state와 pooler_output을 출력
# last_hiden_state : 배치의 각 시퀀스에서 각 토큰에 대한 숨겨진 표현(batch_size, seq_len, hidden_size)
# pooler_output : 배치의 각 시퀀스의 표현을 하나의 벡터로 요약, [CLS] 토큰 사용(batch_size, hidden_size)
class CustomModel(nn.Module):
  def __init__(self, model, num_classes):
    super(CustomModel, self).__init__() # 부모 클래스 초기화
    self.bert = model
    self.dropout = nn.Dropout(p=0.5)
    self.fc = nn.Linear(768, num_classes)

  def forward(self, input_ids, attention_mask, token_type_ids):
    output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    last_hidden_state = output['last_hidden_state']
    output = self.dropout(last_hidden_state[:, 0, :])
    output = self.fc(output)
    return output

In [45]:
kobert = CustomModel(model, num_classes=4)
kobert.to(device)

CustomModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
run = wandb.init(project='kobert')

In [ ]:
def model_train(model, data_loader, optimizer, criterion):
  model. train() # 훈련 모드로 설정
  total_loss = 0
  total_correct = 0
  for inputs, labels in tqdm(data_loader):
    inputs = {k:v.to(device) for k,v in inputs.items()}
    labels = labels.to(device)

    optimizer.zero_grad() # gradient 초기화
    output = model(**inputs)
    loss = criterion(output, labels)
    loss.backward() # 역전파
    optimizer.step() # 가중치 업데이트
    total_loss += loss.item()
    wandb.log({"Batch Loss": loss.item()})

    _, predicted = torch.max(output, dim=1)
    total_correct += (predicted == labels).sum().item()
  accuracy = total_correct / len(data_loader.dataset)
  wandb.log({"Batch Accuracy": accuracy})

  avg_loss = total_loss / len(data_loader.dataset)
  wandb.log({"Average Training Loss": avg_loss})

  return avg_loss, accuracy

In [ ]:
def model_evaluate(model, data_loader, criterion):
  model.eval() # 평가모드로 설정
  with torch.no_grad():
    total_loss = 0
    total_correct = 0
    for inputs, labels in tqdm(data_loader):
      inputs = {k:v.to(device) for k,v in inputs.items()}
      labels = labels.to(device)

      output = model(**inputs)
      _, predicted = torch.max(output, dim=1)
      total_correct += (predicted == labels).sum().item()
      total_loss += criterion(output, labels).item() * labels.size(0)
  accuracy = total_correct / len(data_loader.dataset)
  wandb.log({"Accuracy": accuracy})

  avg_loss = total_loss / len(data_loader.dataset)
  wandb.log({"Average Validation Loss": avg_loss})
  return avg_loss, accuracy

In [ ]:
def run_model(train_loader, val_loader, EPOCHS):
  optimizer = optim.AdamW(kobert.parameters(), lr=1e-5)
  criterion = nn.CrossEntropyLoss()
  min_loss = np.inf
  for epoch in range(EPOCHS):
    avg_train_loss, train_acc = model_train(kobert, train_loader, optimizer, criterion)
    avg_val_loss, val_acc = model_evaluate(kobert, val_loader, criterion)

    if avg_val_loss < min_loss:
      min_loss = avg_val_loss
      torch.save(kobert.state_dict(), 'best_model.pt')
    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_acc:.4f},Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

In [ ]:
run_model(train_loader, val_loader, EPOCHS=10)

100%|██████████| 30/30 [00:08<00:00,  3.50it/s]


Epoch 1/10, Train Loss: 0.0165, Train Accuracy: 0.9643,Val Loss: 0.3313, Val Accuracy: 0.8958


100%|██████████| 30/30 [00:08<00:00,  3.53it/s]


Epoch 2/10, Train Loss: 0.0103, Train Accuracy: 0.9787,Val Loss: 0.2798, Val Accuracy: 0.9167


100%|██████████| 30/30 [00:08<00:00,  3.50it/s]


...

Epoch 8/10, Train Loss: 0.0018, Train Accuracy: 0.9981,Val Loss: 0.2694, Val Accuracy: 0.9083


100%|██████████| 30/30 [00:08<00:00,  3.52it/s]


Epoch 9/10, Train Loss: 0.0022, Train Accuracy: 0.9981,Val Loss: 0.2661, Val Accuracy: 0.9417


100%|██████████| 30/30 [00:08<00:00,  3.53it/s]

Epoch 10/10, Train Loss: 0.0106, Train Accuracy: 0.9750,Val Loss: 0.3254, Val Accuracy: 0.9250


In [50]:
kobert.load_state_dict(torch.load('/content/drive/MyDrive/SeSAC/DL_Project/kobert.pt'))

<All keys matched successfully>

In [51]:
class model_predict():
  def __init__(self, model, tokenizer, labels:dict):
    self.model = model
    self.tokenizer = tokenizer
    self.labels = labels

  def predict(self, sentence):
    tokens = self.tokenizer(sentence,
                            return_tensors='pt',
                            truncation=True,
                            add_special_tokens=True,
                            padding='max_length')
    tokens.to(device)
    prediction = self.model(**tokens)
    prediction = nn.functional.softmax(prediction, dim=1)
    output = prediction.argmax(dim=1).item()
    prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
    return prob, result

In [52]:
labels = {v:k for k,v in label_dict.items()}
predictor = model_predict(kobert, tokenizer, labels)

In [53]:
predictor.predict("대통령 임기 중에 열리는 선거는 ‘정권 평가’일 수밖에 없다. 임기 시작 반년 안쪽에 치러지는 선거라면 대선 승리 기세가 이어지는 덕에 여당이 웬만해서는 지기 어렵다. 하지만 임기 중반에 만나는 선거는 대통령에 대한 중간 평가가 되고 만다. 대통령과 함께 가는 정당인 여당으로서는 평가에 오롯이 묶일 수밖에 없다.윤석열 정부 3년 차에 치러지는 4.10 총선은 중간 평가, 정권 평가의 성격을 피할 수 없다. 야당인 민주당은 평가를 넘어 ‘심판’으로 가야 한다고 외친다. 윤 대통령의 지지율이 높다면야 이 주장은 간단히 일축되겠지만 현실은 그게 아니다. 대개 여론조사에서 지지율(긍정 평가)은 30대 초중반이고 부정 평가가 60%를 넘나든다. 이런 숫자를 한참 전부터 봐왔다.")

(0.9899165034294128, '정치')